# Apresento nesta aula o spaCy,
uma bilbioteca Python para Processamento de Linguagem Natural (NLP). Vamos brincar um pouco com ela, que é bem divertida e já vem com um modelo pronto em português, o que facilita bastante para compreender um pouco o NLP.

Usando um modelo pré-pronto
Podemos usar direto o modelo de português que acabamos de baixar. Para isso, separamos o modelo e chamamos ele genericamente de nlp. Como estamos carregando um modelo já pronto, essa declaração tende a demorar um pouquinho.



In [1]:
import spacy
nlp = spacy.load('pt_core_news_sm')


A partir de agora podemos usar esse modelo para entender frases em português. Vamos usar o nlp para estudar uma frase simples:


In [2]:
doc = nlp(u'Vocês estão gostando da aula, alunos?')


Obs: você deve declarar a string como unicode para que ele funcione corretamente.

# Um pouco sobre docs e tokens…
Um Doc, objeto como aquele que acabamos de criar, é uma sequência de objetos do tipo Token e possui diversas informações sobre o texto que ele contém. Por dividir a frase em tokens, esse documento é uma estrutura iterável e portanto, deve ser acessada como tal. Já um Token é uma parte da estrutura e pode ser uma frase, palavra, uma pontuação, um espaço em branco, etc. No nosso caso, como iremos avaliar uma frase, os tokens serão constituídos de palavras e pontuações.

Primeiro, vamos analisar a frase da maneira mais simples: dividindo-a com o método split de qualquer string.

In [3]:
doc.text.split()

['Vocês', 'estão', 'gostando', 'da', 'aula,', 'alunos?']

Podemos ver que apesar de ser coerente a divisão por espaços, o verbo falei e a vírgula estão dentro de um mesmo token, assim como Carla e a interrogação. O nlp consegue entender a diferença entre eles e, portanto, quando usamos os tokens dentro da estrutura do documento, temos uma divisão mais coerente:

In [4]:
[token for token in doc]

[Vocês, estão, gostando, da, aula, ,, alunos, ?]

Repare que agora a estrutura considerou a pontução e as palavras como estruturas separadas. Também não temos mais uma lista de strings, mas uma lista de Tokens.
Se não quisermos os objetos Tokens, mas sim as strings que cada Token contém podemos usar o método .orth_:

In [5]:
[token.orth_ for token in doc]

['Vocês', 'estão', 'gostando', 'da', 'aula', ',', 'alunos', '?']

# Entendendo diferença entre palavras e pontuações
Como o spaCy entende que existe uma diferença entre uma palavra e uma pontuação, também podemos fazer filtragens. E se eu quisesse apenas as palavras da frase?

In [6]:
[token.orth_ for token in doc if not token.is_punct]

['Vocês', 'estão', 'gostando', 'a', 'aula', 'alunos']

# Similaridade
O spaCy também permite avaliar similaridade entre palavras. O método .similarity de um Token avalia a similaridade semântica estimada entre as palavras. Quanto maior o valor, mais similar são as palavras.

Vamos avaliar a similaridade entre 3 palavras: você, livro e eu.

Primeiro vamos armazenar o tokens em uma lista para acessá-los de forma independente:

In [18]:
doc = nlp(u'Alunos estão gostando da aula, alunos?')
tokens = [token for token in doc]

Dessa forma, temos que tokens[0] representa o meu token da palavra Você e tokens[5] representa a palavra eu. A análise de similaridade pode ser dada por:

In [19]:
print(tokens[0].similarity(tokens[4]))
print(tokens[0],tokens[4])

0.15754366
Alunos aula


/home/brunafrade/.local/lib/python3.6/site-packages/ipykernel_launcher.py:1: UserWarning: [W007] The model you're using has no word vectors loaded, so the result of the Token.similarity method will be based on the tagger, parser and NER, which may not give useful similarity judgements. This may happen if you're using one of the small models, e.g. `en_core_web_sm`, which don't ship with word vectors and only use context-sensitive tensors. You can always add your own word vectors, or use one of the larger models instead if available.
  """Entry point for launching an IPython kernel.


Legal, temos um valor de 0.25802183. E o que isso significa? 

In [20]:
print(tokens[0].similarity(tokens[6]))
tokens[0], tokens[6]

0.7143049


/home/brunafrade/.local/lib/python3.6/site-packages/ipykernel_launcher.py:1: UserWarning: [W007] The model you're using has no word vectors loaded, so the result of the Token.similarity method will be based on the tagger, parser and NER, which may not give useful similarity judgements. This may happen if you're using one of the small models, e.g. `en_core_web_sm`, which don't ship with word vectors and only use context-sensitive tensors. You can always add your own word vectors, or use one of the larger models instead if available.
  """Entry point for launching an IPython kernel.


(Alunos, alunos)

O valor é maior, ou seja, Alunos de fato está semânticamente muito mais próximo de alunos do que de aula, o que faz todo sentido, certo?

Análise de classes gramaticais - etiquetamento morfologico
Podemos também entender as classes gramaticais de cada palavra dentro do nosso contexto:

In [21]:
[(token.orth_, token.pos_) for token in doc]

[('Alunos', 'NOUN'),
 ('estão', 'AUX'),
 ('gostando', 'VERB'),
 ('da', 'DET'),
 ('aula', 'NOUN'),
 (',', 'PUNCT'),
 ('alunos', 'NOUN'),
 ('?', 'PUNCT')]

Então assim conseguimos ver que encontrou e falei são os verbos da frase. E a vírgula e o ponto de interrogação foi corretamente definido como pontuação (PUNCT).
Encontrei, encontraram, encontrarão, encontrariam….
Agora, imagine que você tem um texto enorme e diversos tempos verbais diferentes. A análise passa a ser infinitamente mais complicada! O que podemos fazer, então, é analisar não o verbo no tempo verbal que ele foi escrito, mas ele em sua raiz. O nome desse método de encontrar a raiz das palavras é lematização por isso, o método .lemma_ faz exatamente isso. Vamos olhar:

In [22]:
[token.lemma_ for token in doc if token.pos_ == 'VERB']

['gostar']

E isso vale para diversos tempos verbais MESMO!

In [25]:
doc = nlp(u'encontrei, encontraram, encontrarão, encontrariam')
[token.lemma_ for token in doc if token.pos_ == 'VERB']

['encontrar', 'encontrar', 'encontrar', 'encontrar']

# Ancestrais
Do mesmo jeito que podemos encontrar as raízes de uma palavra, podemos checar se uma palavra é raíz de outra:

In [23]:
doc = nlp(u'encontrar encontrei')
tokens = [token for token in doc]
tokens[0].is_ancestor(tokens[1])

True

E por fim… entidades
Por fim, podemos avaliar as entidades presentes em uma frase. Por exemplo, peguemos a frase:

In [24]:
doc = nlp(u'Machado de Assis um dos melhores escritores do Brasil, foi o primeiro presidente da Academia Brasileira de Letras')

Se formos analisar as entidades presentes nessa frase, percebemos que temos 3 entidades identificadas automaticamente:

In [25]:
doc.ents


(Machado de Assis, Brasil, Academia Brasileira de Letras)

Ao analisarmos detalhadamente, podemos ver que o Spacy identificou Machado de Assis como uma pessoa (PER de person, em inglês), Brasil como um local (LOC) e Academia Brasileira de Letras como uma organização (ORG).

In [ ]:
[(entity, entity.label_) for entity in doc.ents]


Claro que em inglês o modelo está bem avançado, e consegue identificar entidades bem mais complexas do que eu pude verificar nos textos em português. Por exemplo:

In [ ]:
wiki_obama = """Barack Obama is an American politician who served as
     the 44th President of the United States from 2009 to 2017. He is the first
     African American to have served as president,
     as well as the first born outside the contiguous United States."""
nlp = spacy.load('en')
nlp_obama = nlp(wiki_obama)
[(i, i.label_) for i in nlp_obama.ents]

That's all folks!